### 1. Import required packages and setup the Parcl Labs API key

In [ ]:
# if needed, install and/or upgrade to the latest verison of the Parcl Labs Python library
%pip install --upgrade parcllabs==1.10.0 nbformat

In [23]:
import os
import pandas as pd
import plotly.express as px
from datetime import datetime
import plotly.graph_objects as go
from parcllabs import ParclLabsClient
from parcllabs.beta.charting.styling import SIZE_CONFIG
from parcllabs.beta.ts_stats import TimeSeriesAnalysis
from parcllabs.beta.charting.utils import create_labs_logo_dict
from parcllabs.beta.charting.default_charts import create_dual_axis_chart
from parcllabs.beta.charting.styling import default_style_config as style_config


client = ParclLabsClient(
    api_key=os.environ.get('PARCL_LABS_API_KEY', "<your Parcl Labs API key if not set as environment variable>"), 
    limit=12 # set default limit
)

### 2. Search for markets

In [24]:
# get us benchmark
metros = client.search.markets.retrieve(
    sort_by='TOTAL_POPULATION',
    query=', Fl',
    sort_order='DESC',
    location_type='CBSA',
    limit=300 # get top 300 metros based on population
)

# add us national to comp markets against national numbers
us = client.search.markets.retrieve(
    query='United States',
    limit=1
)

markets = pd.concat([metros, us])
market_ids = markets['parcl_id'].unique().tolist()
markets['state'] = markets['name'].apply(lambda x: x.split(',')[-1].strip().upper().split('-')[0])
markets['clean_name'] = markets.apply(lambda x: f"{x['name'].split('-')[0].split(',')[0].strip()}, {x['state']}", axis=1)
markets['clean_name'] = markets['clean_name'].replace({'United States Of America, UNITED STATES OF AMERICA': 'USA'})

In [25]:
markets = markets.loc[markets['total_population']>= 150000]

In [26]:
# Metro parcl id
market_parcl_ids = markets['parcl_id'].tolist()

### 3. Retrieve the Data

In [27]:
# Lets get YoY changes for tampa market -- warning, you need a starter account to do this work. 
# this will pull back a lot of data and use a lot of credits. We are 
# capturing weekly supply numbers for 100 metros across the country. 

start_date='2022-09-01'

supply = client.for_sale_market_metrics.for_sale_inventory.retrieve(
    property_type='CONDO',
    parcl_ids=market_parcl_ids,
    start_date=start_date,
    limit=200
)

demand = client.market_metrics.housing_event_counts.retrieve(
    property_type='CONDO',
    parcl_ids=market_parcl_ids,
    start_date=start_date,
    limit=200
)

prices = client.market_metrics.housing_event_prices.retrieve(
    property_type='CONDO',
    parcl_ids=market_parcl_ids,
    start_date='2019-01-01',
    limit=200
)

In [28]:
# get supply side of the market
total_supply = client.for_sale_market_metrics.for_sale_inventory.retrieve(
    property_type='ALL_PROPERTIES',
    parcl_ids=market_parcl_ids,
    start_date=start_date,
    limit=200
)

In [ ]:
total_supply = total_supply.rename(columns={'for_sale_inventory':'total_supply'})
total_supply = total_supply.merge(supply, on=['parcl_id', 'date'], how='inner')
total_supply['pct_condo_supply'] = total_supply['for_sale_inventory'] / total_supply['total_supply']
total_supply = total_supply.merge(markets[['parcl_id', 'clean_name']], on='parcl_id')

total_supply.head()

In [30]:
# get price changing dynamics
price_changes = client.for_sale_market_metrics.for_sale_inventory_price_changes.retrieve(
    property_type='CONDO',
    parcl_ids=market_parcl_ids,
    limit=200,
    start_date=start_date
)

### 4. Initial data preparation

In [31]:
# supply is a weekly series while demand is a monthly series, need to truncate supply to the month
# and then compress it to a monthly series. We will take the median value of the month to represent
# that month
supply_monthly = supply.copy(deep=True)
supply_monthly = supply_monthly.merge(price_changes[['parcl_id', 'date', 'count_price_drop']], on=['parcl_id', 'date'])
supply_monthly['pct_price_drops'] = supply_monthly['count_price_drop']/supply_monthly['for_sale_inventory']
supply_monthly['date'] = supply_monthly['date'].dt.to_period('M').dt.to_timestamp()
supply_monthly = supply_monthly.groupby(['date', 'parcl_id'])[['for_sale_inventory', 'pct_price_drops']].median().reset_index()


In [ ]:
# now we can join with demand, which is a monthly series
data = pd.merge(demand[['date', 'parcl_id', 'sales']], supply_monthly[['date', 'parcl_id', 'for_sale_inventory']], on=['date', 'parcl_id'])
data

### 5. Supply & demand skew

In [ ]:
data = data.sort_values(['parcl_id', 'date'])
# Calculate percentage change in 'sales' for each 'parcl_id' over 12 periods
data['pct_change_demand'] = data.groupby('parcl_id')['sales'].pct_change(periods=12)
# Calculate percentage change in 'for_sale_inventory' for each 'parcl_id' over 12 periods
data['pct_change_supply'] = data.groupby('parcl_id')['for_sale_inventory'].pct_change(periods=12)
# calculate a 3 month moving average to account for spikiness in the data
data['ma_pct_change_demand'] = data.groupby('parcl_id')['pct_change_demand'].transform(lambda x: x.rolling(window=3).mean())
# Calculate a 3-month moving average for 'pct_change_supply'
data['ma_pct_change_supply'] = data.groupby('parcl_id')['pct_change_supply'].transform(lambda x: x.rolling(window=3).mean())
data.tail()

In [34]:
# clean up the data due to needing 12 observations to account for starting the percent change over a 1 year period
supply_demand_imbalance = data.dropna()

In [35]:
markets['state'] = markets['name'].apply(lambda x: x.split(',')[-1].strip().upper().split('-')[0])
markets['clean_name'] = markets.apply(lambda x: f"{x['name'].split('-')[0].split(',')[0].strip()}, {x['state']}", axis=1)
markets['clean_name'] = markets['clean_name'].replace({'United States Of America, UNITED STATES OF AMERICA': 'USA'})

In [36]:
# Assuming supply_demand_imbalance and metros dataframes are already defined
last_date = supply_demand_imbalance['date'].max()
supply_demand_imbalance_last = supply_demand_imbalance.loc[supply_demand_imbalance['date'] == last_date]
supply_demand_imbalance_last = pd.merge(supply_demand_imbalance_last, markets[['parcl_id', 'clean_name', 'state']], on='parcl_id')
supply_demand_imbalance_last_us = supply_demand_imbalance_last.loc[supply_demand_imbalance_last['parcl_id']==us['parcl_id'].values[0]]
# supply_demand_imbalance_last = supply_demand_imbalance_last.loc[
#     (supply_demand_imbalance_last['sales'] > 500) & 
#     (supply_demand_imbalance_last['for_sale_inventory'] > 500) & 
#     (supply_demand_imbalance_last['ma_pct_change_demand'] < -0.1) & 
#     (supply_demand_imbalance_last['ma_pct_change_supply'] > 0.2)
# ]
supply_demand_imbalance_last = pd.concat([supply_demand_imbalance_last_us, supply_demand_imbalance_last])
# capture parcl_ids for later analysis
imbalanced_parcl_ids = supply_demand_imbalance_last['parcl_id'].unique().tolist()

In [ ]:
markets.loc[markets['parcl_id'].isin(imbalanced_parcl_ids)][['clean_name', 'parcl_id']]

In [ ]:
supply_demand_imbalance_last.loc[supply_demand_imbalance_last['parcl_id']==us['parcl_id'].values[0]]

In [ ]:
# Add a column to identify selected states
target_states = {'FL'}
supply_demand_imbalance_last['color_group'] = supply_demand_imbalance_last['state'].apply(lambda x: 'Florida' if x in target_states else 'Other')

chart_max_date = supply_demand_imbalance_last['date'].max()
chart_max_date = chart_max_date.strftime('%B, %Y')

# Creating the scatter plot
fig = px.scatter(
    supply_demand_imbalance_last, 
    x='ma_pct_change_demand', 
    y='ma_pct_change_supply', 
    color='color_group',  # Use the new color_group column for color
    hover_name='clean_name', 
    title=f'YoY Changes in Supply vs. Demand: Florida Condos ({chart_max_date})',
    color_discrete_map={'Florida': 'red', 'Other': 'blue'},  # Customize colors,
    text='clean_name'
)

fig.update_traces(
    textposition='top center',
    mode='markers+text'  # Ensure that both markers and text are displayed
)

fig.add_layout_image(
        create_labs_logo_dict()
    )

# Update axes labels and layout to format as a square
fig.update_layout(
    margin=dict(l=40, r=40, t=80, b=40),
    title={
        'y': 0.98,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': style_config['title_font']
    },
     xaxis=dict(
            title_text='YoY % Change Demand (Sales)',
            showgrid=style_config['showgrid'],
            gridwidth=style_config['gridwidth'],
            gridcolor=style_config['grid_color'],
            # tickangle=style_config['tick_angle'],
            tickformat='.0%',
            linecolor=style_config['line_color_axis'],
            linewidth=style_config['linewidth'],
            titlefont=style_config['title_font_axis']
        ),
        yaxis=dict(
            title_text='YoY % Change Supply',
            showgrid=style_config['showgrid'],
            gridwidth=style_config['gridwidth'],
            gridcolor=style_config['grid_color'],
            tickfont=style_config['axis_font'],
            zeroline=False,
            tickformat='.0%',
            linecolor=style_config['line_color_axis'],
            linewidth=style_config['linewidth'],
            titlefont=style_config['title_font_axis']
        ),
    plot_bgcolor=style_config['background_color'],
    paper_bgcolor=style_config['background_color'],
    font=dict(color=style_config['font_color']),
    legend_title_text='',
    autosize=False,
    width=1000,
    height=1000,
    title_font=dict(size=24),
    xaxis_title_font=dict(size=18),
    yaxis_title_font=dict(size=18),
    legend_title_font=dict(size=14),
    legend_font=dict(size=12),
    legend=dict(
            x=style_config['legend_x'],
            y=style_config['legend_y'],
            xanchor=style_config['legend_xanchor'],
            yanchor=style_config['legend_yanchor'],
            font=style_config['legend_font'],
            bgcolor='rgba(0, 0, 0, 0)'
        ),
)

fig.show()


In [ ]:
# Sort the data by supply percentage change
supply_demand_imbalance_last = supply_demand_imbalance_last.sort_values(by='ma_pct_change_supply', ascending=True)

chart_max_date = supply_demand_imbalance_last['date'].max()
chart_max_date = chart_max_date.strftime('%B, %Y')

# Prepare the data for the bar chart
data_for_bar = pd.melt(supply_demand_imbalance_last, id_vars=['clean_name'], 
                       value_vars=['ma_pct_change_demand', 'ma_pct_change_supply'], 
                       var_name='type', value_name='percent_change')
data_for_bar['type'] = data_for_bar['type'].map({'ma_pct_change_demand': 'Demand', 'ma_pct_change_supply': 'Supply'})

# Apply bold formatting and light blue color to markets ending with "FL"
data_for_bar['clean_name'] = data_for_bar['clean_name'].apply(lambda x: f"<b style='color:red'>{x}</b>" if x.endswith('FL') else x)

# Create the bar chart
fig = px.bar(data_for_bar, 
             x='clean_name', 
             y='percent_change', 
             color='type', 
             barmode='relative', 
             title=f'YoY Change in Supply and Demand: Florida Condos ({chart_max_date})',
             labels={'percent_change': 'Percent Change', 'clean_name': 'Market'},
             color_discrete_map={'Demand': 'red', 'Supply': 'green'})

# Update the legend names
for trace in fig.data:
    if trace.name == 'Demand':
        trace.name = 'Demand (Sales)'
    elif trace.name == 'Supply':
        trace.name = 'Supply (Inventory)'

fig.update_layout(
    margin=dict(l=40, r=40, t=80, b=40),
    title={
        'y': 0.98,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': style_config['title_font']
    },
    xaxis=dict(
        title_text='',
        showgrid=style_config['showgrid'],
        gridwidth=style_config['gridwidth'],
        gridcolor=style_config['grid_color'],
        linecolor=style_config['line_color_axis'],
        linewidth=style_config['linewidth'],
        titlefont=style_config['title_font_axis'],
        tickfont=dict(size=style_config['axis_font']['size'], color=style_config['axis_font']['color']),
        # showticklabels=False
    ),
    yaxis=dict(
        title_text='Percent Change',
        showgrid=style_config['showgrid'],
        gridwidth=style_config['gridwidth'],
        gridcolor=style_config['grid_color'],
        tickfont=style_config['axis_font'],
        zeroline=False,
        tickformat='.0%',
        linecolor=style_config['line_color_axis'],
        linewidth=style_config['linewidth'],
        titlefont=style_config['title_font_axis']
    ),
    plot_bgcolor=style_config['background_color'],
    paper_bgcolor=style_config['background_color'],
    font=dict(color=style_config['font_color']),
    legend_title_text='',
    autosize=False,
    width=1600,
    height=800,
    title_font=dict(size=24),
    xaxis_title_font=dict(size=18),
    yaxis_title_font=dict(size=18),
    legend_title_font=dict(size=14),
    legend_font=dict(size=12),
    legend=dict(
        x=style_config['legend_x'],
        y=style_config['legend_y'],
        xanchor=style_config['legend_xanchor'],
        yanchor=style_config['legend_yanchor'],
        font=style_config['legend_font'],
        bgcolor='rgba(0, 0, 0, 0)'
    ),
)

fig.add_layout_image(create_labs_logo_dict())

fig.show()


In [ ]:
# supply_demand_imbalance_last = supply_demand_imbalance_last.merge(markets[['parcl_id', 'name']], on='parcl_id')
# supply_demand_imbalance_last[['parcl_id', 'clean_name', 'date', 'pct_change_demand', 'pct_change_supply', 'ma_pct_change_demand', 'ma_pct_change_supply']].to_csv('fl_condo_supply_demand_shifts.csv', index=False)

### 6. New construction impact on supply

In [ ]:
# event counts for new listings
new_listings = client.market_metrics.housing_event_counts.retrieve(
    parcl_ids=imbalanced_parcl_ids,
    property_type='CONDO',
    limit=1
)

# we will need to secure data from 3 separate endpoints
nc_listings = client.new_construction_metrics.housing_event_counts.retrieve(
    parcl_ids=imbalanced_parcl_ids,
    property_type='CONDO',
    limit=1
)

# rename new_listings_for_sale to new_construction
nc_listings = nc_listings.rename(columns={
    'new_listings_for_sale': 'new_construction_new_listings_for_sale'
})

In [ ]:
new_listings = new_listings.merge(nc_listings[['parcl_id', 'new_construction_new_listings_for_sale']], on='parcl_id')
new_listings['pct_new_construction'] = new_listings['new_construction_new_listings_for_sale']/new_listings['new_listings_for_sale']

new_listings = new_listings.merge(markets[['parcl_id', 'clean_name']], on='parcl_id')

In [ ]:
# Prepare the data for the bar chart
new_listings = new_listings.sort_values('pct_new_construction', ascending=True)

chart_max_date = new_listings['date'].max()
chart_max_date = chart_max_date.strftime('%B, %Y')

data_for_bar = pd.melt(new_listings, id_vars=['clean_name'], 
                       value_vars=['pct_new_construction'], 
                       var_name='type', value_name='percentage')
data_for_bar['type'] = data_for_bar['type'].map({'pct_new_construction': 'New Construction'})
data_for_bar = data_for_bar.loc[data_for_bar['type']=='New Construction']

# Apply bold formatting and red color to markets ending with "FL"
data_for_bar['clean_name'] = data_for_bar['clean_name'].apply(lambda x: f"<b style='color:red'>{x}</b>" if x.endswith('FLZ') else x)

# Create the stacked bar chart
fig = px.bar(data_for_bar, 
             x='clean_name', 
             y='percentage', 
             color='type', 
             barmode='stack', 
             title=f'Percent of New Listings Coming from New Construction: Florida Condos ({chart_max_date})',
             labels={'percentage': 'Percentage', 'clean_name': 'Market'},
             color_discrete_map={'New Construction': 'orange', 'Investors': 'blue'})

# Update the legend names
for trace in fig.data:
    if trace.name == 'New Construction':
        trace.name = 'New Construction'
    elif trace.name == 'Investors':
        trace.name = 'Investors'

fig.update_layout(
    margin=dict(l=40, r=40, t=80, b=40),
    title={
        'y': 0.98,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': style_config['title_font']
    },
    xaxis=dict(
        title_text='',
        showgrid=style_config['showgrid'],
        gridwidth=style_config['gridwidth'],
        gridcolor=style_config['grid_color'],
        linecolor=style_config['line_color_axis'],
        linewidth=style_config['linewidth'],
        titlefont=style_config['title_font_axis'],
        tickfont=dict(size=style_config['axis_font']['size'], color=style_config['axis_font']['color']),
        # showticklabels=False
    ),
    yaxis=dict(
        title_text='% of New Inventory',
        showgrid=style_config['showgrid'],
        gridwidth=style_config['gridwidth'],
        gridcolor=style_config['grid_color'],
        tickfont=style_config['axis_font'],
        zeroline=False,
        tickformat='.0%',
        linecolor=style_config['line_color_axis'],
        linewidth=style_config['linewidth'],
        titlefont=style_config['title_font_axis']
    ),
    plot_bgcolor=style_config['background_color'],
    paper_bgcolor=style_config['background_color'],
    font=dict(color=style_config['font_color']),
    legend_title_text='',
    autosize=False,
    width=1600,
    height=800,
    title_font=dict(size=24),
    xaxis_title_font=dict(size=18),
    yaxis_title_font=dict(size=18),
    legend_title_font=dict(size=14),
    legend_font=dict(size=12),
    legend=dict(
        x=style_config['legend_x'],
        y=style_config['legend_y'],
        xanchor=style_config['legend_xanchor'],
        yanchor=style_config['legend_yanchor'],
        font=style_config['legend_font'],
        bgcolor='rgba(0, 0, 0, 0)'
    ),
)

fig.add_layout_image(create_labs_logo_dict())

fig.show()

In [ ]:
# new_listings = new_listings.merge(markets[['parcl_id', 'name']], on='parcl_id')
new_listings[['parcl_id', 'clean_name', 'date', 'property_type', 'pct_new_construction']].to_csv('pct_new_construction.csv', index=False)

### 7. Active supply price drops

Now within these skewed markets, which markets also are having price changes? These markets would now have not only a supply/demand skew but also a supply side that is demonstrating a willingness to sell, actively reducing prices. 

Let's look for markets where at least 35% of the inventory is experiencing price changes.

In [ ]:
supply_monthly['ma_price_changes'] = supply_monthly.groupby('parcl_id')['pct_price_drops'].transform(lambda x: x.rolling(window=3).mean())
s1 = supply_monthly.loc[(supply_monthly['ma_price_changes']>0.35) & (supply_monthly['date']=='6/1/2024')].sort_values('ma_price_changes', ascending=False)
s1 = s1.loc[s1['parcl_id'].isin(imbalanced_parcl_ids)]
imbalanced_with_price_changes_pids = s1['parcl_id'].unique().tolist()

In [ ]:
# markets left
markets.loc[markets['parcl_id'].isin(imbalanced_with_price_changes_pids)][['clean_name']]

In [ ]:
price_changes_skewed = price_changes.merge(supply[['parcl_id', 'date', 'for_sale_inventory']], on=['parcl_id', 'date'])
price_changes_skewed['pct_price_drops'] = price_changes_skewed['count_price_drop']/price_changes_skewed['for_sale_inventory']
price_changes_skewed = price_changes_skewed.merge(markets[['parcl_id', 'clean_name']], on='parcl_id')

max_date_for_chart = price_changes_skewed['date'].max().date()
max_date_for_chart = max_date_for_chart.strftime('%B %d, %Y')

# Create the line chart using Plotly Express
fig = px.line(
    price_changes_skewed,
    x='date',
    y='pct_price_drops',
    color='clean_name',
    line_group='clean_name',
    labels={'pct_price_drops': '% of Inventory with Price Cuts'},
    title=f'Percentage of Inventory with Price Reductions: Florida Condos ({max_date_for_chart})'
)

# Update traces to apply specific styles
for trace in fig.data:
    if trace.name == 'USA':
        trace.update(
            line=dict(color='red', width=4),
            opacity=1
        )
    else:
        trace.update(
            line=dict(color='lightblue', dash='dash', width=2),
            opacity=0.8
        )
    # Remove text annotations from traces
    trace.update(
        mode='lines'
    )

# Find the latest date in the dataset
latest_date = max(price_changes_skewed['date'])

# Add annotations for each line on the far right
annotations = []
y_positions = []

for trace in fig.data:
    # Get the last y-value for each clean_name
    last_y_value = price_changes_skewed[
        (price_changes_skewed['clean_name'] == trace.name) &
        (price_changes_skewed['date'] == latest_date)
    ]['pct_price_drops'].values[0]
    
    # Only add the annotation if it doesn't overlap with existing annotations
    if not any(abs(last_y_value - y) < 0.01 for y in y_positions):  # Adjust threshold as needed
        annotations.append(dict(
            x=latest_date,
            y=last_y_value,
            xref='x',
            yref='y',
            text=trace.name,
            showarrow=False,
            xanchor='left',
            font=dict(size=12)  # Adjust the font size if needed
        ))
        y_positions.append(last_y_value)

fig.add_layout_image(
        create_labs_logo_dict()
)

# Update layout for axes, title, and other styling
fig.update_layout(
    width=1600,
    height=800,
    xaxis=dict(
        title='',
        showgrid=style_config['showgrid'],
        gridwidth=style_config['gridwidth'],
        gridcolor=style_config['grid_color'],
        # tickangle=style_config['tick_angle'],
        linecolor=style_config['line_color_axis'],
        linewidth=style_config['linewidth'],
        titlefont=style_config['title_font_axis']
    ),
    yaxis=dict(
        title='% Price Reductions',
        showgrid=style_config['showgrid'],
        gridwidth=style_config['gridwidth'],
        gridcolor=style_config['grid_color'],
        tickfont=style_config['axis_font'],
        zeroline=False,
        tickformat='.0%',
        linecolor=style_config['line_color_axis'],
        linewidth=style_config['linewidth'],
        titlefont=style_config['title_font_axis']
    ),
    plot_bgcolor=style_config['background_color'],
    paper_bgcolor=style_config['background_color'],
    font=dict(color=style_config['font_color']),
    showlegend=False,  # Remove the legend
    margin=dict(l=40, r=40, t=80, b=40),
    title={
        'y': 0.98,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(size=24)
    },
    annotations=annotations  # Add annotations
)

fig.show()


In [ ]:
#price_changes_skewed = price_changes_skewed.merge(markets[['parcl_id', 'name']], on='parcl_id')
price_changes_skewed[['parcl_id', 'clean_name', 'date', 'pct_price_drops']].to_csv('fl_condo_pct_inventory_with_price_drops_weekly.csv', index=False)

In [ ]:
price_changes_skewed.loc[price_changes_skewed['date']=='8/5/2024'].sort_values('pct_price_drops', ascending=False)

In [ ]:
# percent of total supply that are condos
max_date_for_chart = total_supply['date'].max().date()
max_date_for_chart = max_date_for_chart.strftime('%B %d, %Y')

# Create the line chart using Plotly Express
fig = px.line(
    total_supply,
    x='date',
    y='pct_condo_supply',
    color='clean_name',
    line_group='clean_name',
    labels={'pct_condo_supply': '% of Inventory from Condos'},
    title=f'Condo Share of All Active Inventory ({max_date_for_chart})'
)

# Update traces to apply specific styles
for trace in fig.data:
    if trace.name == 'USA':
        trace.update(
            line=dict(color='red', width=4),
            opacity=1
        )
    else:
        trace.update(
            line=dict(color='lightblue', dash='dash', width=2),
            opacity=0.8
        )
    # Remove text annotations from traces
    trace.update(
        mode='lines'
    )

# Find the latest date in the dataset
latest_date = max(total_supply['date'])

# Add annotations for each line on the far right
annotations = []
y_positions = []

for trace in fig.data:
    # Get the last y-value for each clean_name
    last_y_value = total_supply[
        (total_supply['clean_name'] == trace.name) &
        (total_supply['date'] == latest_date)
    ]['pct_condo_supply'].values[0]
    
    # Only add the annotation if it doesn't overlap with existing annotations
    if not any(abs(last_y_value - y) < 0.01 for y in y_positions):  # Adjust threshold as needed
        annotations.append(dict(
            x=latest_date,
            y=last_y_value,
            xref='x',
            yref='y',
            text=trace.name,
            showarrow=False,
            xanchor='left',
            font=dict(size=12)  # Adjust the font size if needed
        ))
        y_positions.append(last_y_value)

fig.add_layout_image(
        create_labs_logo_dict()
)

# Update layout for axes, title, and other styling
fig.update_layout(
    width=1600,
    height=800,
    xaxis=dict(
        title='',
        showgrid=style_config['showgrid'],
        gridwidth=style_config['gridwidth'],
        gridcolor=style_config['grid_color'],
        # tickangle=style_config['tick_angle'],
        linecolor=style_config['line_color_axis'],
        linewidth=style_config['linewidth'],
        titlefont=style_config['title_font_axis']
    ),
    yaxis=dict(
        title='% Condo',
        showgrid=style_config['showgrid'],
        gridwidth=style_config['gridwidth'],
        gridcolor=style_config['grid_color'],
        tickfont=style_config['axis_font'],
        zeroline=False,
        tickformat='.0%',
        linecolor=style_config['line_color_axis'],
        linewidth=style_config['linewidth'],
        titlefont=style_config['title_font_axis']
    ),
    plot_bgcolor=style_config['background_color'],
    paper_bgcolor=style_config['background_color'],
    font=dict(color=style_config['font_color']),
    showlegend=False,  # Remove the legend
    margin=dict(l=40, r=40, t=80, b=40),
    title={
        'y': 0.98,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(size=24)
    },
    annotations=annotations  # Add annotations
)

fig.show()


### 8. Appreciation since COVID

In [ ]:
# filter to most out of balance markets regarding supply and demand
prices_skew = prices.loc[prices['parcl_id'].isin(imbalanced_with_price_changes_pids + [5826765])]

all_rows = []
for pid in prices_skew['parcl_id'].unique().tolist():
    prices_skew_test = prices_skew.loc[prices_skew['parcl_id']==pid]
    price_ts_analysis = TimeSeriesAnalysis(prices_skew_test, 'date', 'price_per_square_foot_median_sales', freq='M')
    price_rate_of_change_stats = price_ts_analysis.calculate_changes(change_since_date='3/1/2020')
    row = pd.json_normalize(price_rate_of_change_stats)
    row['parcl_id'] = pid
    all_rows.append(row)

In [ ]:
ts_analysis = pd.concat(all_rows)

In [ ]:
hf = ts_analysis.loc[(ts_analysis['peak_to_current.percent_change'] > -.05) & (ts_analysis['change_since_date.percent_change']>.5)]

In [ ]:
# markets left
hf[['parcl_id' ,'peak_to_current.percent_change', 'change_since_date.percent_change']].merge(markets[['parcl_id', 'clean_name']], on='parcl_id')

In [ ]:
hf_ids = hf['parcl_id'].unique().tolist()
prices_skew = prices.loc[prices['parcl_id'].isin(hf_ids + [5826765])]


In [ ]:
# show percent change relative to first value
prices_skew = prices.copy(deep=True)
prices_skew_chart = prices_skew.loc[prices_skew['date']>='3/1/2020']
prices_skew_chart = prices_skew_chart.sort_values('date')
prices_skew_chart = prices_skew_chart[['date', 'parcl_id', 'price_per_square_foot_median_sales']]
prices_skew_chart_first = prices_skew_chart.loc[prices_skew_chart['date']=='3/1/2020'].rename(columns={'price_per_square_foot_median_sales':'start'})
chart = pd.merge(prices_skew_chart, prices_skew_chart_first[['parcl_id', 'start']], on='parcl_id')
chart['pct_change'] = (chart['price_per_square_foot_median_sales']-chart['start'])/chart['start']
chart = chart.merge(markets[['parcl_id', 'clean_name']], on='parcl_id')

In [ ]:

# get max date
chart_max_date = chart['date'].max()
chart_max_date = chart_max_date.strftime('%B, %Y')

fig = px.line(
    chart,
    x='date',
    y='pct_change',
    color='clean_name',
    line_group='clean_name',
    labels={'pct_change': '% Change'},
    title=f'% Change in Condo Values since the Start of the Pandemic ({chart_max_date})'
)

# Update traces to apply specific styles
for trace in fig.data:
    if trace.name == 'USA':
        trace.update(
            line=dict(color='red', width=4),
            opacity=1
        )
    else:
        trace.update(
            line=dict(color='lightblue', dash='dash', width=2),
            opacity=0.8
        )
    # Remove text annotations from traces
    trace.update(
        mode='lines'
    )

# Find the latest date in the dataset
latest_date = max(chart['date'])

# Add annotations for each line on the far right
annotations = []
y_positions = []

for trace in fig.data:
    # Get the last y-value for each clean_name
    last_y_value = chart[
        (chart['clean_name'] == trace.name) &
        (chart['date'] == latest_date)
    ]['pct_change'].values[0]
    
    # Only add the annotation if it doesn't overlap with existing annotations
    if not any(abs(last_y_value - y) < 0.02 for y in y_positions):  # Adjust threshold as needed
        annotations.append(dict(
            x=latest_date,
            y=last_y_value,
            xref='x',
            yref='y',
            text=trace.name,
            showarrow=False,
            xanchor='left',
            font=dict(size=12)  # Adjust the font size if needed
        ))
        y_positions.append(last_y_value)

fig.add_layout_image(
        create_labs_logo_dict()
)

# Update layout for axes, title, and other styling
fig.update_layout(
    width=1600,
    height=800,
    xaxis=dict(
        title='',
        showgrid=style_config['showgrid'],
        gridwidth=style_config['gridwidth'],
        gridcolor=style_config['grid_color'],
        # tickangle=style_config['tick_angle'],
        linecolor=style_config['line_color_axis'],
        linewidth=style_config['linewidth'],
        titlefont=style_config['title_font_axis']
    ),
    yaxis=dict(
        title='% Change',
        showgrid=style_config['showgrid'],
        gridwidth=style_config['gridwidth'],
        gridcolor=style_config['grid_color'],
        tickfont=style_config['axis_font'],
        zeroline=False,
        tickformat='.0%',
        linecolor=style_config['line_color_axis'],
        linewidth=style_config['linewidth'],
        titlefont=style_config['title_font_axis']
    ),
    plot_bgcolor=style_config['background_color'],
    paper_bgcolor=style_config['background_color'],
    font=dict(color=style_config['font_color']),
    showlegend=False,  # Remove the legend
    margin=dict(l=40, r=40, t=80, b=40),
    title={
        'y': 0.98,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(size=24)
    },
    annotations=annotations  # Add annotations
)

fig.show()
